<a href="https://colab.research.google.com/github/Harshkotkar/Deep-Learning/blob/main/APTOS_Eye_Preprocessing_in_Diabetic_Retinopathy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [4]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aptos2019_blindness_detection_path = kagglehub.competition_download('aptos2019-blindness-detection')

print('Data source import complete.')


Data source import complete.


In [5]:
 # This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import os

print(aptos2019_blindness_detection_path)  # This shows the base folder
print(os.listdir(aptos2019_blindness_detection_path))  # List what’s inside


/root/.cache/kagglehub/competitions/aptos2019-blindness-detection
['train_images', 'train.csv', 'test.csv', 'test_images', 'sample_submission.csv']


In [7]:
import os
import pandas as pd

# Base dataset path
base_path = "/root/.cache/kagglehub/competitions/aptos2019-blindness-detection"

# Load CSVs
train_df = pd.read_csv(os.path.join(base_path, "train.csv"))
test_df = pd.read_csv(os.path.join(base_path, "test.csv"))

print(train_df.head())
print(test_df.head())


        id_code  diagnosis
0  000c1434d8d7          2
1  001639a390f0          4
2  0024cdab0c1e          1
3  002c21358ce6          0
4  005b95c28852          0
        id_code
0  0005cfc8afb6
1  003f0afdcd15
2  006efc72b638
3  00836aaacf06
4  009245722fa4


In [8]:
train_images_path = os.path.join(base_path, "train_images")
test_images_path  = os.path.join(base_path, "test_images")


###Why tf.data? Efficient handling of large image datasets → parallelism + prefetching.

Why resize to (224×224)? Most pretrained CNN backbones (ResNet, EfficientNet, Inception) expect 224×224 inputs. Keeping consistency allows us to later plug in pretrained models easily.

Why normalize (0–1)? Neural nets converge faster when input values are small and consistent. Images originally have pixel values 0–255; dividing by 255 standardizes them. New section

In [9]:
import tensorflow as tf
import os

train_imgs = "/root/.cache/kagglehub/competitions/aptos2019-blindness-detection/train_images"

IMG_SIZE = 224
BATCH_SIZE = 32

def load_image(img_id, label):
    # Create image path using tf.strings.join
    img_path = tf.strings.join([train_imgs, "/", img_id, ".png"])

    # Read image file
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0  # normalize

    return img, label

# Convert dataset
train_ds = tf.data.Dataset.from_tensor_slices((train_df['id_code'].values,
                                               train_df['diagnosis'].values))
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


### Train Test Split

##why do we not do the X_train ,Y_train, that one split?
### Our Case (APTOS Dataset)

Our dataframe looks like this:

id_code         	diagnosis
000c1434d8d7	       2
001639a390f0	       4
0024cdab0c1e	       1
id_code → not really a feature, it’s just the filename of the image.

diagnosis → the label (0–4).


##In this dataset, the dataframe doesn’t directly contain the features (images).

##Instead, it maps id_code (filename) → diagnosis (label).

##We split the dataframe rows while keeping both columns, then later load the actual images using those filenames.

###Why stratify?
Because the dataset is imbalanced (some disease classes are rarer). stratify ensures both train and validation have similar class distributions.

In [10]:
from sklearn.model_selection import train_test_split

# Train-validation split
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['diagnosis'], random_state=42)

print("Training samples:", len(train_df))
print("Validation samples:", len(val_df))


Training samples: 2929
Validation samples: 733


In [11]:
!ls /root/.cache/kagglehub/competitions/aptos2019-blindness-detection/train_images | head -20


000c1434d8d7.png
001639a390f0.png
0024cdab0c1e.png
002c21358ce6.png
005b95c28852.png
0083ee8054ee.png
0097f532ac9f.png
00a8624548a9.png
00b74780d31d.png
00cb6555d108.png
00cc2b75cddd.png
00e4ddff966a.png
00f6c1be5a33.png
0104b032c141.png
0124dffecf29.png
0125fbd2e791.png
012a242ac6ff.png
014508ccb9cb.png
0151781fe50b.png
0161338f53cc.png


In [15]:
# train_imgs = "/root/.cache/kagglehub/competitions/aptos2019-blindness-detection/train_images"

# # Fix filenames to match actual files
# train_df["id_code"] = train_df["id_code"].astype(str) + ".png"
# test_df["id_code"]  = test_df["id_code"].astype(str) + ".png"

# # ImageDataGenerator
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# train_datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

# train_generator = train_datagen.flow_from_dataframe(
#     dataframe=train_df,
#     directory="/root/.cache/kagglehub/competitions/aptos2019-blindness-detection/train_images",
#     x_col="id_code",
#     y_col="diagnosis",
#     subset="training",
#     batch_size=32,
#     seed=42,
#     shuffle=True,
#     class_mode="categorical",
#     target_size=(224, 224))

# valid_generator = train_datagen.flow_from_dataframe(
#     dataframe=train_df,
#     directory="/root/.cache/kagglehub/competitions/aptos2019-blindness-detection/train_images",
#     x_col="id_code",
#     y_col="diagnosis",
#     subset="validation",
#     batch_size=32,
#     seed=42,
#     shuffle=True,
#     class_mode="categorical",
#     target_size=(224, 224))



In [16]:
# Fix filenames first (only one .png at the end)
train_df["id_code"] = train_df["id_code"].str.replace(r"(\.png)+$", ".png", regex=True)
test_df["id_code"]  = test_df["id_code"].str.replace(r"(\.png)+$", ".png", regex=True)

# Train-validation split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['diagnosis'], random_state=42)

# Define dataset loader
def load_image(img_id, label):
    img_path = tf.strings.join([train_imgs, "/", img_id])
    img = tf.io.read_file(img_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = img / 255.0
    return img, label

# Create datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_df["id_code"].values, train_df["diagnosis"].values))
train_ds = train_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).shuffle(1024).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_df["id_code"].values, val_df["diagnosis"].values))
val_ds = val_ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [18]:
import os

train_imgs = "/root/.cache/kagglehub/competitions/aptos2019-blindness-detection/train_images"
files = os.listdir(train_imgs)
print("Total files in train_images:", len(files))
print("First 10 files:", files[:10])


Total files in train_images: 3662
First 10 files: ['a3b2e93d058b.png', '936299166bea.png', '5b3d41626ec5.png', '172df1330a60.png', '25d069089c5e.png', '504a69096fcb.png', 'cd5714db652d.png', 'cd1c98ec48b1.png', '7f1f3269f546.png', 'a30a143a53a3.png']


In [17]:
# Check if all image names exist in train_images folder
missing_files = [f for f in train_df["id_code"].values if f not in files]
print("Missing files:", len(missing_files))
print(missing_files[:10])


Missing files: 0
[]


In [ ]:
from tensorflow.keras import layers, models

# Simple CNN Model

model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(5, activation='softmax')
])



In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
#conv2d_4
#Kh=kernal heingt,Kw=kernal width, Cin=chanel input as rgb it  is 3 and grayscal is 1
#(Kh x Kw x Cin x Cout )+Cout
#(3×3×32×64)+64=(18,432)+64=18,496

In [ ]:
history=model.fit(train_ds,
                  validation_data=val_ds,
                  epochs=10,
                  verbose=1)